In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns

font = {'size'   : 12}

matplotlib.rc('font', **font)
matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
matplotlib.rc('text', usetex=True)


# Load Results

In [ ]:
df = pd.read_csv('results.csv')
df.head()

# Define Plotting Function

In [ ]:
# This code was largely takken from https://gist.github.com/davidandrzej/939840 (Thanks, David!)

import matplotlib.ticker as MT
import matplotlib.lines as L
import matplotlib.cm as CM
import matplotlib.colors as C
import matplotlib.patches as PA

def plotSimplex(points,colors,titles,**kwargs):
    """
    Plot 4xNx3 points array on the 3-simplex 
    
    kwargs will be passed along directly to matplotlib.pyplot.scatter    
    Returns Figure, caller must .show()
    """
    # 4 panels for conditions
    offsets = [(0,1.3),(1.3,1.3),(0,0),(1.3,0)]
    
    fig = plt.figure()
    
    for point,color,title,(xoff,yoff) in zip(points,colors,titles,offsets):
        # Draw the triangle
        l1 = L.Line2D([0+xoff, 0.5+xoff, 1+xoff, 0+xoff], # xcoords
                      [0+yoff, np.sqrt(3) / 2+yoff, 0+yoff, 0+yoff], # ycoords
                      color='k')
        fig.gca().add_line(l1)
        fig.gca().xaxis.set_major_locator(MT.NullLocator())
        fig.gca().yaxis.set_major_locator(MT.NullLocator())
        # Draw vertex labels
        fig.gca().text(0+xoff, -0.15+yoff, r'$\overline{UI}_X$')
        fig.gca().text(0.86+xoff, -0.15+yoff, r'$\overline{UI}_Y$')
        fig.gca().text(0.43+xoff, np.sqrt(3) / 2 + 0.05+yoff, r'$\overline{RI}$')
        fig.gca().text(0.05+xoff, np.sqrt(3) / 2 -0.2 +yoff,title)
        # Project and draw the actual points
        projected = projectSimplex(point)
        plt.scatter(projected[:,0]+xoff, projected[:,1]+yoff,c=color, **kwargs)              
        # Leave some buffer around the triangle for vertex labels
        fig.gca().set_xlim(-0.195, 2.45)
        fig.gca().set_ylim(-0.22, 2.4)
        
    cbar = plt.colorbar()
    cbar.set_label(r'$\overline{SI}$',rotation=0)
    plt.clim([0,1])

    return fig    

def projectSimplex(points):
    """ 
    Project probabilities on the 3-simplex to a 2D triangle
    
    N points are given as N x 3 array
    """
    # Convert points one at a time
    tripts = np.zeros((points.shape[0],2))
    for idx in range(points.shape[0]):
        # Init to triangle centroid
        x = 1.0 / 2
        y = 1.0 / (2 * np.sqrt(3))
        # Vector 1 - bisect out of lower left vertex 
        p1 = points[idx, 0]
        x = x - (1.0 / np.sqrt(3)) * p1 * np.cos(np.pi / 6)
        y = y - (1.0 / np.sqrt(3)) * p1 * np.sin(np.pi / 6)
        # Vector 2 - bisect out of lower right vertex  
        p2 = points[idx, 1]  
        x = x + (1.0 / np.sqrt(3)) * p2 * np.cos(np.pi / 6)
        y = y - (1.0 / np.sqrt(3)) * p2 * np.sin(np.pi / 6)        
        # Vector 3 - bisect out of top vertex
        p3 = points[idx, 2]
        y = y + (1.0 / np.sqrt(3) * p3)
      
        tripts[idx,:] = (x,y)

    return tripts

# Plot Results

In [ ]:
# get samples for condition 1
S1 = df.loc[(df['dm'] == df['dx']) & (df['dm'] == df['dy'])]
# get samples for condition 2
S2 = df.loc[(df['dm'] < df['dx']) & (df['dm'] < df['dy']) ]
# get samples for condition 3
S3 = df.loc[(df['dm'] > df['dx']) & (df['dm'] > df['dy']) ]
# get samples for condition 4
S4 = df.loc[((df['dm'] < df['dx']) & (df['dm'] > df['dy'])) | ((df['dm'] > df['dx']) & (df['dm'] < df['dy'])) ]

# store the positions and colors for each condition
points = []
colors = []
for dfs in [S1,S2,S3,S4]:
    points.append(np.vstack((dfs['nuix'].values,dfs['nuiy'].values,dfs['nri'].values)).T)
    colors.append(dfs['nsi'].values)

# titles for panels
titles = [r'$(S1)$',r'$(S2)$',r'$(S3)$',r'$(S4)$']
    
# define colormap
cmap = CM.get_cmap('nipy_spectral')

# do scatter plot
fig = plotSimplex(points,colors,titles,s=0.3,cmap=cmap)

# save figure
plt.savefig('simplex.png',dpi=500)